In [1]:
from unsloth import FastLanguageModel

Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-09-11 14:16:30.838869: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-11 14:16:30.864961: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-11 14:16:31.292884: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
max_seq_length = 2048

dtype = None

load_in_4bit = True

# 모델 설정
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="./q_blos",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    cache_dir='/data'
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.66 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.24. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv('./empathy1.csv', encoding='UTF-8')

dataset = Dataset.from_pandas(df)

dataset

Dataset({
    features: ['user0', 'user1'],
    num_rows: 17168
})

In [5]:
dataset['user0'][0]

'우리 딸, 엄마가 보험 만기가 되어서 환급을 받았는데 액수가 꽤 많아. 어찌나 좋은지 웃음이 절로 난다. 이 보험금 넣는다고 힘들었는데 지금은 잘했다 싶은 게 뿌듯하다.'

In [6]:
def formatting_empathy_prompts_func(examples):
    output_texts = []
    for i in range(len(examples['user0'])):
        messages = [
            {"role": "system", "content": "공감하는 챗봇으로서 상대방의 입력에 대해 공감을 하자. 모든 대답은 한국어(Korean)으로 대답해줘."},
            {"role": "user", "content": "{}".format(examples['user0'][i])},
            {"role": "assistant", "content": "{}".format(examples['user1'][i])}
        ]
        # LLAMA의 모델 클래스에 맞는 기본 채팅 템플릿 적용
        chat_message = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        output_texts.append(chat_message)

    return {"text": output_texts}

In [7]:
formatted_dataset = dataset.map(formatting_empathy_prompts_func, batched=True)

Map:   0%|          | 0/17168 [00:00<?, ? examples/s]

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,  # 멀티프로세싱을 위해 더 많은 프로세스를 사용 (코랩에서 사용할 수 있는 리소스에 따라 조정)
    packing=False,  # 짧은 시퀀스의 경우 False가 더 나을 수 있음
    args=TrainingArguments(
        per_device_train_batch_size=2,  # GPU 메모리에 맞춰 배치 크기 조정
        gradient_accumulation_steps=4,  # 배치 크기를 효과적으로 늘리기 위해 그래디언트 누적 사용
        warmup_steps=100,  # 워밍업 스텝을 늘려 모델이 더 안정적으로 학습하도록 설정
        max_steps=1000,  # 학습 스텝 수를 늘려 충분한 학습 시간 확보
        learning_rate=5e-5,  # 보편적으로 사용되는 학습률
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps=50,  # 로그를 덜 빈번하게 출력하여 학습 속도에 영향을 주지 않도록 함
        optim="adamw_torch",  # 최신 옵티마이저 사용 (PyTorch의 기본 AdamW 구현)
        weight_decay=0.01,  # 일반적으로 사용되는 weight decay
        lr_scheduler_type="cosine",  # 학습률이 부드럽게 감소하도록 cosine 스케줄 사용
        seed=3407,
        output_dir="outputs",
        save_steps=100,  # 체크포인트를 자주 저장하여 중간에 학습을 중단하고도 재개 가능하도록 설정
    ),
)

trainer_stats = trainer.train()

Map (num_proc=2):   0%|          | 0/17168 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 17,168 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,000
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
50,3.441300
100,2.068600
150,1.884700
200,1.760100
250,1.744000
300,1.703900
350,1.691200
400,1.653500
450,1.662100
500,1.665700


In [10]:
model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 44.91 out of 62.61 RAM for saving.


100%|██████████| 32/32 [00:09<00:00,  3.34it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at model into bf16 GGUF format.
The output location will be ./model/unsloth.BF16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,        

: 